In [21]:
%cd /Users/eric/code/ml-misc/feast_tutorial/feature_repo/

/Users/eric/code/ml-misc/feast_tutorial/feature_repo


/Users/eric/Library/Caches/pypoetry/virtualenvs/ml-misc-v-Xd_ym0-py3.9/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [22]:
import pandas as pd

In [23]:
features = pd.read_parquet("./data/driver_stats.parquet")
features.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2025-05-01 19:58:41.023304+00:00,1001,1.000000,1.000000,1000,2025-05-01 19:58:41.023305
1,2025-05-01 18:37:19.316620+00:00,1001,1.000000,1.000000,1000,2025-05-01 18:37:19.316622
2,2025-04-16 18:00:00+00:00,1005,0.110278,0.872387,777,2025-05-01 18:28:46.036000
3,2025-04-16 19:00:00+00:00,1005,0.528918,0.794825,157,2025-05-01 18:28:46.036000
4,2025-04-16 20:00:00+00:00,1005,0.159503,0.415134,971,2025-05-01 18:28:46.036000


In [24]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        # values we're using for an on-demand transformation
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

store = FeatureStore(repo_path=".")

/Users/eric/Library/Caches/pypoetry/virtualenvs/ml-misc-v-Xd_ym0-py3.9/lib/python3.9/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


In [25]:
store

FeatureStore(
    repo_path=PosixPath('.'),
    config=RepoConfig(project='feast_tutorial', provider='local', registry_config='data/registry.db', online_config={'type': 'sqlite', 'path': 'data/online_store.db'}, auth={'type': 'no_auth'}, offline_config='dask', batch_engine_config='local', feature_server=None, flags=None, repo_path=PosixPath('.'), entity_key_serialization_version=2, coerce_tz_aware=True),
    registry=<feast.infra.registry.registry.Registry object at 0x16a609310>,
    provider=<feast.infra.passthrough_provider.PassthroughProvider object at 0x177e6c910>
)

In [26]:
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

print("----- Feature schema -----\n")
print(training_df.info())

print()
print("----- Example features -----\n")
print(training_df.head())

----- Feature schema -----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   driver_id                           3 non-null      int64              
 1   event_timestamp                     3 non-null      datetime64[ns, UTC]
 2   label_driver_reported_satisfaction  3 non-null      int64              
 3   val_to_add                          3 non-null      int64              
 4   val_to_add_2                        3 non-null      int64              
 5   conv_rate                           3 non-null      float32            
 6   acc_rate                            3 non-null      float32            
 7   avg_daily_trips                     3 non-null      int32              
 8   conv_rate_plus_val1                 3 non-null      float64            
 9   conv_rate_plus_val2

In [27]:
type(training_df)

pandas.core.frame.DataFrame

In [28]:
entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

print("\n----- Example features -----\n")
print(training_df.head())


----- Example features -----

   driver_id                  event_timestamp  \
0       1003 2025-05-02 00:26:38.068305+00:00   
1       1002 2025-05-02 00:26:38.068305+00:00   
2       1001 2025-05-02 00:26:38.068305+00:00   

   label_driver_reported_satisfaction  val_to_add  val_to_add_2  conv_rate  \
0                                   3           3            30   0.662988   
1                                   5           2            20   0.119979   
2                                   1           1            10   1.000000   

   acc_rate  avg_daily_trips  conv_rate_plus_val1  conv_rate_plus_val2  
0  0.415009              148             3.662988            30.662988  
1  0.918781               81             2.119979            20.119979  
2  1.000000             1000             2.000000            11.000000  


In [29]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

pprint(feature_vector)

{'acc_rate': [0.9606223702430725, 0.2365042269229889],
 'avg_daily_trips': [365, 112],
 'conv_rate': [0.5189543962478638, 0.7309948801994324],
 'driver_id': [1004, 1005]}


/Users/eric/Library/Caches/pypoetry/virtualenvs/ml-misc-v-Xd_ym0-py3.9/lib/python3.9/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


In [30]:
store.get_online_features?

Signature:
store.get_online_features(
    features: Union[List[str], feast.feature_service.FeatureService],
    entity_rows: Union[List[Dict[str, Any]], Mapping[str, Union[Sequence[Any], Sequence[feast.types.Value_pb2.Value], feast.types.Value_pb2.RepeatedValue]]],
    full_feature_names: bool = False,
) -> feast.online_response.OnlineResponse
Docstring:
Retrieves the latest online feature data.

Note: This method will download the full feature registry the first time it is run. If you are using a
remote registry like GCS or S3 then that may take a few seconds. The registry remains cached up to a TTL
duration (which can be set to infinity). If the cached registry is stale (more time than the TTL has
passed), then a new registry will be downloaded synchronously by this method. This download may
introduce latency to online feature retrieval. In order to avoid synchronous downloads, please call
refresh_registry() prior to the TTL being reached. Remember it is possible to set the cache TTL

In [35]:
from pprint import pprint
from feast import FeatureStore
feature_store = FeatureStore('.')  # Initialize the feature store

feature_service = feature_store.get_feature_service("driver_activity_v1")
feature_vector = feature_store.get_online_features(
    features=feature_service,
    entity_rows=[
        # {join_key: entity_value}
        {
            "driver_id": 1004,
            "val_to_add": 0.0,
            "val_to_add_2": 0.0,
        },
        {
            "driver_id": 1005,
            "val_to_add": 0.0,
            "val_to_add_2": 0.0,
        },
    ],
).to_dict()
pprint(feature_vector)

{'conv_rate': [0.5189543962478638, 0.7309948801994324],
 'conv_rate_plus_val1': [0.5189543962478638, 0.7309948801994324],
 'conv_rate_plus_val2': [0.5189543962478638, 0.7309948801994324],
 'driver_id': [1004, 1005]}
